In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names

In [3]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [4]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"

In [5]:
revenue_band_path = os.path.join(jpg_path, 'revenue_band') + "/"
if not os.path.exists(revenue_band_path):
    os.makedirs(revenue_band_path)

In [6]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [7]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

policy_types = [
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

# Create a DataFrame with decision_variable_names as the index
decision_variable_map = get_decision_variable_map()

In [8]:
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [9]:
print_unique_resources = get_printable_resource_names(unique_resources)

In [10]:
print_unique_resources

['NG CC', 'NG CT', 'Wind', 'Solar', 'BESS']

In [11]:
# intialize dictionary for saving results for each case
skew_dict = {case: {} for case in case_names}
std_dict = {case: {} for case in case_names}
kurtosis_dict = {case: {} for case in case_names}

In [12]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')


    cem_prices, policies_prices_dict, prices_cols = get_case_outputs_all_models(cem_path, 
        policies_path, case_name, 'energy prices', policy_types)
    

    pf_prices = policies_prices_dict['pf']
    dlac_prices = policies_prices_dict['dlac-p']
    dlac_imperfect_prices = policies_prices_dict['dlac-i']
    slac_prices = policies_prices_dict['slac']
    
    # get zone of prices
    zone_name = prices_cols[0]

    # calculate the skew of prices for each model
    cem_skew = cem_prices[zone_name].skew()
    pf_skew = pf_prices[zone_name].skew()
    dlac_skew = dlac_prices[zone_name].skew()
    dlac_imperfect_skew = dlac_imperfect_prices[zone_name].skew()
    slac_skew = slac_prices[zone_name].skew()


    skew_df = {'CEM': cem_skew, 'PF': pf_skew, 'DLAC-p': dlac_skew, 'DLAC-i': dlac_imperfect_skew, 'SLAC': slac_skew}

    # calculate the standard deviation of prices for each model
    cem_std = cem_prices[zone_name].std()
    pf_std = pf_prices[zone_name].std()
    dlac_std = dlac_prices[zone_name].std()
    dlac_imperfect_std = dlac_imperfect_prices[zone_name].std()
    slac_std = slac_prices[zone_name].std()

    std_df = {'CEM': cem_std, 'PF': pf_std, 'DLAC-p': dlac_std, 'DLAC-i': dlac_imperfect_std, 'SLAC': slac_std}

    # save results to dictionary
    skew_dict[case_name] = skew_df
    std_dict[case_name] = std_df

    # calculate kurtosis of prices for each model
    cem_kurtosis = cem_prices[zone_name].kurtosis()
    pf_kurtosis = pf_prices[zone_name].kurtosis()
    dlac_kurtosis = dlac_prices[zone_name].kurtosis()
    dlac_imperfect_kurtosis = dlac_imperfect_prices[zone_name].kurtosis()
    slac_kurtosis = slac_prices[zone_name].kurtosis()

    kurtosis_df = {
        'CEM': cem_kurtosis,
        'PF': pf_kurtosis,
        'DLAC-p': dlac_kurtosis,
        'DLAC-i': dlac_imperfect_kurtosis,
        'SLAC': slac_kurtosis
    }

    kurtosis_dict[case_name] = kurtosis_df
# end of loop    

Case Name: Thermal_Base

Case Name: 2_Hr_BESS

Case Name: 2_Hr_BESS_Fuelx2

Case Name: 4_Hr_BESS

Case Name: 4_Hr_BESS_Fuelx2

Case Name: 4_Hr_BESS_Fuelx3

Case Name: 4_Hr_BESS_Fuelx4

Case Name: 6_Hr_BESS

Case Name: 6_Hr_BESS_Fuelx2

Case Name: 8_Hr_BESS

Case Name: 8_Hr_BESS_Fuelx2

Case Name: 10_Hr_BESS

Case Name: 10_Hr_BESS_Fuelx2



In [13]:
def percent_change_df(df):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df['PF'].astype(float)
    norm['PF'] = df['PF'].astype(float) / df['PF'].astype(float)
    norm['DLAC-p'] = df['DLAC-p'].astype(float) / df['PF'].astype(float)
    norm['DLAC-i'] = df['DLAC-i'].astype(float) / df['PF'].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df['PF'].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC-p'] = norm['DLAC-p'] - 1
    change_df['DLAC-i'] = norm['DLAC-i'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC-p'] = change_df['DLAC-p'] * 100
    percent_change_df['DLAC-i'] = change_df['DLAC-i'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
                        percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].round(2)
    percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
        percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].astype(str) + '%'

    return percent_change_df

In [14]:
std_results_df = pd.DataFrame.from_dict(std_dict, orient='index')
# std_results_df = pd.DataFrame.from_dict(std_dict, orient='index').reset_index()
# std_results_df = std_results_df.rename(columns={'index': 'Case Name'})
# display
std_results_df
# print to csv
std_results_df.to_csv(os.path.join(csv_path, 'Table5a_std_results.csv'))
std_results_df

,CEM,PF,DLAC-p,DLAC-i,SLAC
Thermal_Base,171.081433,178.515473,178.056278,208.053607,172.055316
2_Hr_BESS,273.513444,274.938118,270.567664,248.368946,244.999757
2_Hr_BESS_Fuelx2,251.761883,332.213551,228.786565,236.220308,229.136837
4_Hr_BESS,270.660793,293.880104,290.999441,266.102326,259.556574
4_Hr_BESS_Fuelx2,236.978937,337.165530,254.033686,234.802941,231.276192
4_Hr_BESS_Fuelx3,271.068296,275.615274,257.539614,232.094252,199.361477
4_Hr_BESS_Fuelx4,285.738862,289.637819,264.748343,267.808644,231.088776
6_Hr_BESS,252.561877,299.879903,279.553267,257.843464,259.251838
6_Hr_BESS_Fuelx2,261.770539,272.399097,239.898753,229.465028,224.937401
8_Hr_BESS,246.880117,333.681682,244.772631,273.660832,263.883467


In [15]:
std_rank_df = std_results_df.copy()
models = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
for i, row in std_results_df.iterrows():
    # argsort returns indices of sorted array, so we invert to get descending order
    ranks = (-row[models]).argsort().argsort() + 1
    # Assign ranks to the dataframe
    for idx, model in enumerate(models):
        std_rank_df.at[i, model] = ranks[idx]
std_rank_df[models] = std_rank_df[models].astype(int)
std_rank_df

,CEM,PF,DLAC-p,DLAC-i,SLAC
Thermal_Base,5,2,3,1,4
2_Hr_BESS,2,1,3,4,5
2_Hr_BESS_Fuelx2,2,1,5,3,4
4_Hr_BESS,3,1,2,4,5
4_Hr_BESS_Fuelx2,3,1,2,4,5
4_Hr_BESS_Fuelx3,2,1,3,4,5
4_Hr_BESS_Fuelx4,2,1,4,3,5
6_Hr_BESS,5,1,2,4,3
6_Hr_BESS_Fuelx2,2,1,3,4,5
8_Hr_BESS,4,1,5,2,3


In [16]:
std_results_change_df = percent_change_df(std_results_df)
# print to csv
std_results_change_df.to_csv(os.path.join(csv_path, 'Table5b_std_results_change.csv'))
# print df to latex .tex file
std_results_change_df.to_latex(latex_path + 'std_results_change.tex', 
    index=True, float_format="%.2f", escape=False, column_format='lccccccccc')
# display
std_results_change_df

,CEM,PF,DLAC-p,DLAC-i,SLAC
Thermal_Base,-4.16%,0.0%,-0.26%,16.55%,-3.62%
2_Hr_BESS,-0.52%,0.0%,-1.59%,-9.66%,-10.89%
2_Hr_BESS_Fuelx2,-24.22%,0.0%,-31.13%,-28.9%,-31.03%
4_Hr_BESS,-7.9%,0.0%,-0.98%,-9.45%,-11.68%
4_Hr_BESS_Fuelx2,-29.71%,0.0%,-24.66%,-30.36%,-31.41%
4_Hr_BESS_Fuelx3,-1.65%,0.0%,-6.56%,-15.79%,-27.67%
4_Hr_BESS_Fuelx4,-1.35%,0.0%,-8.59%,-7.54%,-20.21%
6_Hr_BESS,-15.78%,0.0%,-6.78%,-14.02%,-13.55%
6_Hr_BESS_Fuelx2,-3.9%,0.0%,-11.93%,-15.76%,-17.42%
8_Hr_BESS,-26.01%,0.0%,-26.64%,-17.99%,-20.92%


In [17]:

# create table of skew results with case names as index and models as columns
# Convert the skew_dict to a DataFrame
skew_results_df = pd.DataFrame.from_dict(skew_dict, orient='index').reset_index()
skew_results_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# print to csv
skew_results_df.to_csv(os.path.join(csv_path, 'Table5c_skew_results.csv'), index=False)

# Display the DataFrame
skew_results_df


,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,26.066425,25.199410,25.416166,22.141328,25.584114
1,2_Hr_BESS,16.931639,16.915360,17.102639,18.639200,18.699731
2,2_Hr_BESS_Fuelx2,16.090825,13.939071,19.989363,19.207830,18.722906
3,4_Hr_BESS,16.702564,16.430046,16.649293,18.098820,18.117996
4,4_Hr_BESS_Fuelx2,13.620444,13.881548,17.564000,19.451664,18.621899
5,4_Hr_BESS_Fuelx3,16.000618,15.711814,16.522980,18.351048,19.430940
6,4_Hr_BESS_Fuelx4,15.567894,15.046748,15.934882,16.226156,16.088728
7,6_Hr_BESS,16.865545,15.787578,16.744327,18.627680,18.323367
8,6_Hr_BESS_Fuelx2,16.385872,16.347215,17.744747,19.721008,19.130260
9,8_Hr_BESS,16.801498,14.127274,17.819046,17.734437,17.945932


In [18]:
# Create a dataframe of ranks (1 = highest skew, 5 = lowest) for each case and model
skew_rank_df = skew_results_df.copy()
models = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
for i, row in skew_results_df.iterrows():
    # argsort returns indices of sorted array, so we invert to get descending order
    ranks = (-row[models]).argsort().argsort() + 1
    # Assign ranks to the dataframe
    for idx, model in enumerate(models):
        skew_rank_df.at[i, model] = ranks[idx]
skew_rank_df[models] = skew_rank_df[models].astype(int)
skew_rank_df


,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,1,4,3,5,2
1,2_Hr_BESS,4,5,3,2,1
2,2_Hr_BESS_Fuelx2,4,5,1,2,3
3,4_Hr_BESS,3,5,4,2,1
4,4_Hr_BESS_Fuelx2,5,4,3,1,2
5,4_Hr_BESS_Fuelx3,4,5,3,2,1
6,4_Hr_BESS_Fuelx4,4,5,3,1,2
7,6_Hr_BESS,3,5,4,1,2
8,6_Hr_BESS_Fuelx2,4,5,3,1,2
9,8_Hr_BESS,4,5,2,3,1


In [19]:
skew_results_change_df = percent_change_df(skew_results_df)
# print df to latex .tex file
skew_results_change_df.to_latex(latex_path + 'skew_results_change.tex', 
    index=True, float_format="%.2f", escape=False, column_format='lccccccccc')
skew_results_change_df

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,3.44%,0.0%,0.86%,-12.14%,1.53%
1,2_Hr_BESS,0.1%,0.0%,1.11%,10.19%,10.55%
2,2_Hr_BESS_Fuelx2,15.44%,0.0%,43.41%,37.8%,34.32%
3,4_Hr_BESS,1.66%,0.0%,1.33%,10.16%,10.27%
4,4_Hr_BESS_Fuelx2,-1.88%,0.0%,26.53%,40.13%,34.15%
5,4_Hr_BESS_Fuelx3,1.84%,0.0%,5.16%,16.8%,23.67%
6,4_Hr_BESS_Fuelx4,3.46%,0.0%,5.9%,7.84%,6.92%
7,6_Hr_BESS,6.83%,0.0%,6.06%,17.99%,16.06%
8,6_Hr_BESS_Fuelx2,0.24%,0.0%,8.55%,20.64%,17.02%
9,8_Hr_BESS,18.93%,0.0%,26.13%,25.53%,27.03%


In [20]:
kurtosis_results_df = pd.DataFrame.from_dict(kurtosis_dict, orient='index').reset_index()
kurtosis_results_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# print to csv
kurtosis_results_df.to_csv(os.path.join(csv_path, 'Table5d_kurtosis_results.csv'), index=False)
# Display the DataFrame
kurtosis_results_df

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,735.196690,685.455647,693.525426,518.177683,720.225423
1,2_Hr_BESS,289.447810,289.110867,296.163253,352.693929,356.471570
2,2_Hr_BESS_Fuelx2,283.858946,197.261310,425.244053,386.752052,377.222080
3,4_Hr_BESS,281.838028,270.069663,277.358576,329.604876,332.657722
4,4_Hr_BESS_Fuelx2,190.253969,194.949001,325.873086,396.375197,370.977615
5,4_Hr_BESS_Fuelx3,272.475709,264.305288,298.341541,369.679541,441.913147
6,4_Hr_BESS_Fuelx4,258.495629,244.820801,283.212184,288.728138,308.608390
7,6_Hr_BESS,299.493632,253.100919,287.697402,350.660931,340.257949
8,6_Hr_BESS_Fuelx2,281.269369,280.142458,341.872338,410.243267,395.015655
9,8_Hr_BESS,304.673981,200.160554,339.897356,315.478902,326.661690


In [21]:
kurtosis_rank_df = kurtosis_results_df.copy()
models = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
for i, row in kurtosis_results_df.iterrows():
    # argsort returns indices of sorted array, so we invert to get descending order
    ranks = (-row[models]).argsort().argsort() + 1
    # Assign ranks to the dataframe
    for idx, model in enumerate(models):
        kurtosis_rank_df.at[i, model] = ranks[idx]
kurtosis_rank_df[models] = kurtosis_rank_df[models].astype(int)
kurtosis_rank_df
# print kurtosis results to csv


,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,1,4,3,5,2
1,2_Hr_BESS,4,5,3,2,1
2,2_Hr_BESS_Fuelx2,4,5,1,2,3
3,4_Hr_BESS,3,5,4,2,1
4,4_Hr_BESS_Fuelx2,5,4,3,1,2
5,4_Hr_BESS_Fuelx3,4,5,3,2,1
6,4_Hr_BESS_Fuelx4,4,5,3,2,1
7,6_Hr_BESS,3,5,4,1,2
8,6_Hr_BESS_Fuelx2,4,5,3,1,2
9,8_Hr_BESS,4,5,1,3,2


In [22]:
kurtosis_change_df = percent_change_df(kurtosis_results_df)
# print df to latex .tex file
kurtosis_change_df.to_latex(latex_path + 'kurtosis_results_change.tex', 
    index=True, float_format="%.2f", escape=False, column_format='lccccccccc')
kurtosis_change_df

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,7.26%,0.0%,1.18%,-24.4%,5.07%
1,2_Hr_BESS,0.12%,0.0%,2.44%,21.99%,23.3%
2,2_Hr_BESS_Fuelx2,43.9%,0.0%,115.57%,96.06%,91.23%
3,4_Hr_BESS,4.36%,0.0%,2.7%,22.04%,23.17%
4,4_Hr_BESS_Fuelx2,-2.41%,0.0%,67.16%,103.32%,90.29%
5,4_Hr_BESS_Fuelx3,3.09%,0.0%,12.88%,39.87%,67.2%
6,4_Hr_BESS_Fuelx4,5.59%,0.0%,15.68%,17.93%,26.05%
7,6_Hr_BESS,18.33%,0.0%,13.67%,38.55%,34.44%
8,6_Hr_BESS_Fuelx2,0.4%,0.0%,22.04%,46.44%,41.01%
9,8_Hr_BESS,52.21%,0.0%,69.81%,57.61%,63.2%


In [23]:
# create a latex table that includes 3 master columns for std, skew, and kurtosis
# and in each master column include sub columns for each model
# Prepare dataframes for multi-index columns
std_df_disp = std_results_df.copy()
skew_df_disp = skew_results_df.copy()
kurtosis_df_disp = kurtosis_results_df.copy()

# Set index to 'Case Name' for all
std_df_disp = std_df_disp.set_index('Case Name') if 'Case Name' in std_df_disp.columns else std_df_disp
skew_df_disp = skew_df_disp.set_index('Case Name') if 'Case Name' in skew_df_disp.columns else skew_df_disp
kurtosis_df_disp = kurtosis_df_disp.set_index('Case Name') if 'Case Name' in kurtosis_df_disp.columns else kurtosis_df_disp

# Prepare rank dataframes (already indexed by 'Case Name')
std_rank_disp = std_rank_df.set_index('Case Name') if 'Case Name' in std_rank_df.columns else std_rank_df
skew_rank_disp = skew_rank_df.set_index('Case Name') if 'Case Name' in skew_rank_df.columns else skew_rank_df
kurtosis_rank_disp = kurtosis_rank_df.set_index('Case Name') if 'Case Name' in kurtosis_rank_df.columns else kurtosis_rank_df

# Combine value and rank for each statistic/model
def combine_value_rank(val_df, rank_df):
    out = val_df.copy()
    for model in models:
        out[model] = val_df[model].astype(int).astype(str) + " (" + rank_df[model].astype(str) + ")"
    return out

std_combined = combine_value_rank(std_df_disp[models], std_rank_disp[models])
skew_combined = combine_value_rank(skew_df_disp[models], skew_rank_disp[models])
kurtosis_combined = combine_value_rank(kurtosis_df_disp[models], kurtosis_rank_disp[models])

# Concatenate with MultiIndex columns
tuples = []
for stat in ['Std', 'Skew', 'Kurtosis']:
    for model in models:
        tuples.append((stat, model))
multi_cols = pd.MultiIndex.from_tuples(tuples)

combined = pd.concat(
    [std_combined, skew_combined, kurtosis_combined],
    axis=1
)
combined.columns = multi_cols

# Save to latex
combined.to_latex(
    latex_path + 'Table5_master_std_skew_kurtosis_table_with_ranks.tex',
    index=True,
    escape=False,
    column_format='l' + 'ccccc'*3
)

# # Save to latex using f.write instead of to_latex
# with open(latex_path + 'Table5_master_std_skew_kurtosis_table_with_ranks.tex', 'w', encoding='utf-8') as f:
#     f.write(combined.to_latex(
#         index=True,
#         escape=False,
#         column_format='l' + 'ccccc'*3
#     ))

# Display the combined table
combined

Std                                        Skew  \
                       CEM       PF   DLAC-p   DLAC-i     SLAC     CEM   
Thermal_Base       171 (5)  178 (2)  178 (3)  208 (1)  172 (4)  26 (1)   
2_Hr_BESS          273 (2)  274 (1)  270 (3)  248 (4)  244 (5)  16 (4)   
2_Hr_BESS_Fuelx2   251 (2)  332 (1)  228 (5)  236 (3)  229 (4)  16 (4)   
4_Hr_BESS          270 (3)  293 (1)  290 (2)  266 (4)  259 (5)  16 (3)   
4_Hr_BESS_Fuelx2   236 (3)  337 (1)  254 (2)  234 (4)  231 (5)  13 (5)   
4_Hr_BESS_Fuelx3   271 (2)  275 (1)  257 (3)  232 (4)  199 (5)  16 (4)   
4_Hr_BESS_Fuelx4   285 (2)  289 (1)  264 (4)  267 (3)  231 (5)  15 (4)   
6_Hr_BESS          252 (5)  299 (1)  279 (2)  257 (4)  259 (3)  16 (3)   
6_Hr_BESS_Fuelx2   261 (2)  272 (1)  239 (3)  229 (4)  224 (5)  16 (4)   
8_Hr_BESS          246 (4)  333 (1)  244 (5)  273 (2)  263 (3)  16 (4)   
8_Hr_BESS_Fuelx2   273 (3)  277 (1)  277 (2)  186 (5)  188 (4)  16 (3)   
10_Hr_BESS         233 (4)  283 (1)  213 (5)  250 (2)  246 (3)  14 (4)   
10_Hr_BESS_Fuelx2  277 (3)  280 (1)  279 (2)  204 (4)  199 (5)  16 (3)   

                                                  Kurtosis                    \
                       PF  DLAC-p  DLAC-i    SLAC      CEM       PF   DLAC-p   
Thermal_Base       25 (4)  25 (3)  22 (5)  25 (2)  735 (1)  685 (4)  693 (3)   
2_Hr_BESS          16 (5)  17 (3)  18 (2)  18 (1)  289 (4)  289 (5)  296 (3)   
2_Hr_BESS_Fuelx2   13 (5)  19 (1)  19 (2)  18 (3)  283 (4)  197 (5)  425 (1)   
4_Hr_BESS          16 (5)  16 (4)  18 (2)  18 (1)  281 (3)  270 (5)  277 (4)   
4_Hr_BESS_Fuelx2   13 (4)  17 (3)  19 (1)  18 (2)  190 (5)  194 (4)  325 (3)   
4_Hr_BESS_Fuelx3   15 (5)  16 (3)  18 (2)  19 (1)  272 (4)  264 (5)  298 (3)   
4_Hr_BESS_Fuelx4   15 (5)  15 (3)  16 (1)  16 (2)  258 (4)  244 (5)  283 (3)   
6_Hr_BESS          15 (5)  16 (4)  18 (1)  18 (2)  299 (3)  253 (5)  287 (4)   
6_Hr_BESS_Fuelx2   16 (5)  17 (3)  19 (1)  19 (2)  281 (4)  280 (5)  341 (3)   
8_Hr_BESS          14 (5)  17 (2)  17 (3)  17 (1)  304 (4)  200 (5)  339 (1)   
8_Hr_BESS_Fuelx2   16 (5)  16 (4)  22 (1)  21 (2)  290 (3)  275 (5)  276 (4)   
10_Hr_BESS         14 (5)  18 (1)  17 (2)  17 (3)  222 (4)  211 (5)  362 (1)   
10_Hr_BESS_Fuelx2  16 (5)  16 (4)  21 (1)  19 (2)  287 (3)  274 (5)  276 (4)   

                                     
                    DLAC-i     SLAC  
Thermal_Base       518 (5)  720 (2)  
2_Hr_BESS          352 (2)  356 (1)  
2_Hr_BESS_Fuelx2   386 (2)  377 (3)  
4_Hr_BESS          329 (2)  332 (1)  
4_Hr_BESS_Fuelx2   396 (1)  370 (2)  
4_Hr_BESS_Fuelx3   369 (2)  441 (1)  
4_Hr_BESS_Fuelx4   288 (2)  308 (1)  
6_Hr_BESS          350 (1)  340 (2)  
6_Hr_BESS_Fuelx2   410 (1)  395 (2)  
8_Hr_BESS          315 (3)  326 (2)  
8_Hr_BESS_Fuelx2   570 (1)  508 (2)  
10_Hr_BESS         328 (2)  327 (3)  
10_Hr_BESS_Fuelx2  494 (1)  452 (2)

In [24]:
# remove 'CEM' from combined tables
std_combined_no_cem = std_combined.drop(columns=['CEM'])
skew_combined_no_cem = skew_combined.drop(columns=['CEM'])
kurtosis_combined_no_cem = kurtosis_combined.drop(columns=['CEM'])

In [25]:
# Save each statistic (Std, Skew, Kurtosis) as a separate LaTeX table with value (rank) format

# Std table
std_combined_no_cem.to_latex(
    latex_path + 'Table5_std_with_ranks.tex',
    index=True,
    escape=False,
    column_format='lccccc'
)

# Skew table
skew_combined_no_cem.to_latex(
    latex_path + 'Table5_skew_with_ranks.tex',
    index=True,
    escape=False,
    column_format='lccccc'
)

# Kurtosis table
kurtosis_combined_no_cem.to_latex(
    latex_path + 'Table5_kurtosis_with_ranks.tex',
    index=True,
    escape=False,
    column_format='lccccc'
)

# Display as example
std_combined_no_cem, skew_combined_no_cem, kurtosis_combined_no_cem


(                        PF   DLAC-p   DLAC-i     SLAC
 Thermal_Base       178 (2)  178 (3)  208 (1)  172 (4)
 2_Hr_BESS          274 (1)  270 (3)  248 (4)  244 (5)
 2_Hr_BESS_Fuelx2   332 (1)  228 (5)  236 (3)  229 (4)
 4_Hr_BESS          293 (1)  290 (2)  266 (4)  259 (5)
 4_Hr_BESS_Fuelx2   337 (1)  254 (2)  234 (4)  231 (5)
 4_Hr_BESS_Fuelx3   275 (1)  257 (3)  232 (4)  199 (5)
 4_Hr_BESS_Fuelx4   289 (1)  264 (4)  267 (3)  231 (5)
 6_Hr_BESS          299 (1)  279 (2)  257 (4)  259 (3)
 6_Hr_BESS_Fuelx2   272 (1)  239 (3)  229 (4)  224 (5)
 8_Hr_BESS          333 (1)  244 (5)  273 (2)  263 (3)
 8_Hr_BESS_Fuelx2   277 (1)  277 (2)  186 (5)  188 (4)
 10_Hr_BESS         283 (1)  213 (5)  250 (2)  246 (3)
 10_Hr_BESS_Fuelx2  280 (1)  279 (2)  204 (4)  199 (5),
                        PF  DLAC-p  DLAC-i    SLAC
 Case Name                                        
 Thermal_Base       25 (4)  25 (3)  22 (5)  25 (2)
 2_Hr_BESS          16 (5)  17 (3)  18 (2)  18 (1)
 2_Hr_BESS_Fuelx2   13 (5

In [26]:
def print_price_statistic_latex_table(df, feature, models, latex_path):
    latex_file = os.path.join(latex_path, 'Table5_' + feature + '_with_ranks.tex')
    with open(latex_file, 'w') as f:
        f.write('\\begin{table}[htp]\n')
        f.write('\\centering\n')
        f.write('\\caption{'+ feature + ' of prices (\$/MWh) and their rank order from largest (1) to smallest (5)}\n')
        f.write('\\begin{tabular}{l' + 'c' * len(models) + '}\n')
        f.write('\\hline\n')
        f.write('\\textbf{Case Name} ')
        for col in models:
            f.write(f"& \\multicolumn{{1}}{{l}}{{\\textbf{{{col}}}}} \n")
        f.write('\\\\\n')
        f.write('\\hline\n')
        for index, row in df.iterrows():
            index_name = index.replace('_', ' ')
            f.write(index_name + ' & ' + ' & '.join(row) + ' \\\\\n')
        f.write('\\hline\n')
        f.write('\\end{tabular}\n')
        f.write('\\label{tab:std_with_ranks}\n')
        f.write('\\end{table}\n')

In [27]:
models_no_cem = ['PF', 'DLAC-p', 'DLAC-i', 'SLAC']

In [28]:

# print_price_statistic_latex_table(std_combined_no_cem, 'Standard deviation', 
#                                   models_no_cem, latex_path)
#
# print_price_statistic_latex_table(skew_combined_no_cem, 'Skew',
#                                   models_no_cem, latex_path)
# print_price_statistic_latex_table(kurtosis_combined_no_cem, 'Kurtosis',
#                                   models_no_cem, latex_path)

In [29]:

print_price_statistic_latex_table(std_combined, 'Standard deviation', 
                                  ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC'], latex_path)

print_price_statistic_latex_table(skew_combined, 'Skew',
                                  ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC'] , latex_path)
print_price_statistic_latex_table(kurtosis_combined, 'Kurtosis',
                                  ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC'] , latex_path)